In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os, logging, pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
#from keras import backend as K

## Ignore all the warnings
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_WARNINGS'] = '0'

plt.rcParams.update(plt.rcParamsDefault)

import matplotlib.pyplot as plt




In [19]:
folder_path = r"C:\Users\e0427809\Desktop\Try\well_50\wells\w12_4"  
merged_data = []

#Get the file names starting with "erate" in the folder
file_names = [filename for filename in os.listdir(folder_path) if filename.startswith("erate")]

# Sort filenames based on numerical size
sorted_file_names = sorted(file_names, key=lambda x: int(x.replace("erate", "").replace(".csv", "")))

for file_name in sorted_file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    column_data = df.iloc[:, 1]  # Extract the second column of data
    merged_data.extend(column_data.tolist())  # Add data to merged list
merged_df = pd.DataFrame(merged_data, columns=["w12_4"])
merged_df.to_csv(r"C:\Users\e0427809\Desktop\Try\well_50\wells\w12_4.csv", index=False)

In [4]:
input_file_path = r"C:\Users\e0427809\Desktop\JI_Project/daily_27scenarios.csv"  

df = pd.read_csv(input_file_path)

data = df.iloc[:, 1:].values

merged_data = pd.Series()

for column_data in data.T:
    merged_data = merged_data.append(pd.Series(column_data))

output_df = pd.DataFrame({"Merged Data": merged_data})

output_df.to_csv(r'C:\Users\e0427809\Desktop\JI_Project\prep.csv', index=False)


In [ ]:
root_folder_path = r"C:\Users\e0427809\Desktop\Try\well_50\wells"  # Adjust the root folder path accordingly
output_csv_path = r"C:\Users\e0427809\Desktop\Try\well_50\merged_data.csv"  # Adjust the output CSV file path accordingly
merged_data = []

# Get folder names in the root folder that start with "w"
folder_names = [foldername for foldername in os.listdir(root_folder_path) if foldername.startswith("w")]


for folder_name in folder_names:
    folder_path = os.path.join(root_folder_path, folder_name)
    
    # Get file names in the current folder that start with "erate"
    file_names = [filename for filename in os.listdir(folder_path) if filename.startswith("erate")]
    
    # Sort file names based on numeric order
    sorted_file_names = sorted(file_names, key=lambda x: int(x.replace("erate", "").replace(".csv", "")))
    
    column_data = []
    for file_name in sorted_file_names:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        column_data.extend(df.iloc[:, 1].tolist())  # Append the second column data to the column_data list
    
    merged_data.append(column_data)  # Append the column_data list to the merged_data list

merged_df = pd.DataFrame(merged_data).T  # Transpose the DataFrame to have columns instead of rows
merged_df.columns = folder_names  # Set column names as the sorted folder names

merged_df.to_csv(r'C:\Users\e0427809\Desktop\Try\well_50\merged_data.csv', index=False)

In [5]:
df = pd.read_csv(r'C:\Users\e0427809\Desktop\JI_Project\Freatic_head_20220601000001.csv',header = None)
# Find the index of the row and column that correspond to the given location
lat = 137462.5
lon = 353137.5
lon_idx = np.where(df.iloc[0, :] == lon)[0][0]
lat_idx = np.where(df.iloc[:, 0] == lat)[0][0]
lon_range = range(max(lon_idx - 5, 0), min(lon_idx + 5, len(df.columns)))
lat_range = range(max(lat_idx - 5, 0), min(lat_idx + 5, len(df)))

# extract the values of the 5x5 grid
grid = df.iloc[lat_range, lon_range]

grid.to_csv(r'C:\Users\e0427809\Desktop\JI_Project\Spatial\Freatic_head_well1.csv', index=False,header = False)

In [7]:
bedrock = pd.read_csv(r'C:\Users\e0427809\Desktop\JI_Project\Spatial\bedrock_25_well1.csv')
Freatic_head = pd.read_csv(r'C:\Users\e0427809\Desktop\JI_Project\Spatial\Freatic_head_well1.csv')
khv = pd.read_csv(r'C:\Users\e0427809\Desktop\JI_Project\Spatial\khv_25_well1.csv')

In [8]:
feature1 = Freatic_head.values
feature2 = khv.values
feature3 = bedrock.values


In [9]:
data_mean = feature1.mean(axis=0)
data_std = feature1.std(axis=0)
scaled_data1= (feature1-data_mean)/data_std
data_mean2 = feature2.mean(axis=0)
data_std2 = feature2.std(axis=0)
scaled_data2= (feature2-data_mean2)/data_std2
data_mean3 = feature3.mean(axis=0)
data_std3 = feature3.std(axis=0)
scaled_data3 = (feature3-data_mean3)/data_std3

In [10]:
input_cnn_data = np.stack([scaled_data1, scaled_data2, scaled_data3], axis=-1)
input_cnn_data.shape

x_train_cnn = np.repeat(input_cnn_data, 4409, axis=0)
x_train_cnn = np.reshape(x_train_cnn, (4409, 9, 10, 3))

x_train_cnn.shape